# Конвертация модели HuggingFace в формат ONNX

In [3]:
import numpy as np
import torch
from transformers import AutoTokenizer
from optimum.onnxruntime import (
    AutoOptimizationConfig, ORTOptimizer, ORTModelForSequenceClassification, OptimizationConfig
)

In [2]:
# transformers.__version__ 4.41.2
# onnx.__version__ 1.16.1
# torch.__version__ 2.4.0+cu121
# optimum==1.22.0
# onnxruntime-gpu 1.19.2

## Задание настроек

In [4]:
ORIGINAL_MODEL_PATH = r"C:\RAG\bge-reranker-v2-m3"
ONNX_MODEL_PATH = r"C:\RAG\bge-reranker-v2-m3\onnx\bge-reranker-v2-m3-onnx-gpu"
DEVICE = "cuda"  # Изменено на CUDA
MAX_LENGTH = 8192

In [5]:
# Проверяем, доступна ли CUDA
if torch.cuda.is_available():
    print("CUDA is available")
    print("Number of GPUs: ", torch.cuda.device_count())
else:
    print("CUDA is not available")

CUDA is available
Number of GPUs:  1


In [6]:
test_queries = [['Сколько стоит тариф Премиум??',
  '## Премиум Пакеты на интернет, звонки, SMS\n\nНазвание в ССМ-портале Коммерческое название\nОсобенности\nПерсональная МС\xa0Безлимитный интернет\nБезлимитный интернет Не расходуют пакет интернет-трафика Не работает в МНР Скорость\xa0снижается до 128 кб/сек\xa0на торрент сервисах\nПерсональная МС 3 ГБ + 3 ГБ Персональная МС 5 ГБ + 5 ГБ Персональная МС 15 ГБ + 15 ГБ Персональная МС БВС Безлимит на МегаФон Звонки на МегаФон по России, не расходует пакет минут тарифа\nПерсональная МС 300 минут +300 минут Расходуются на все номера России\nПерсональная МС Безлимитные SMS\nБезлимитные SMS Расходуются на все направления по РФ за исключением номера 900\nПерсональная МС 60 SMS +60 SMS Расходуются на все номера России\n\n## Определение направлений вызовов\nОписание направлений:\nРегион пребывания клиента - регион, из которого клиент совершает звонок, отправляет SMS, выходит в интернет.\xa0\n\nНапример, абонент Новосибирска приехал в Санкт-Петербург. При нахождении в Санкт-Петербурге для такого абонента\xa0**Регион пребывания**\xa0– Санкт- Петербург и Ленинградская область.\n\nМеждугородный звонок - звонок по России за пределы региона пребывания, в т.ч. звонки на номера ЛНР, ДНР, Запорожской и Херсонской обл. Международный\xa0 звонок - звонок за пределы России. Звонки на номера всех операторов России - вызовы на мобильные и городские номера России. Звонки на всех операторов - включает в себя и мобильные и городские. Звонки по всей России - включает в себя звонки по региону пребывания и междугородние.\n## Схема начисления АП и предоставления пакетов\nПросмотр размера АП в ССМ-портале Без информации о названии тарифного плана:\n Перейти в Финансы (1) → Счета (2) далее выбрать период, за который интересует размер абон.платы В нижнем окне закладки Детали можно посмотреть размер АП за выбранный период.\n\nС информацией о тарифном плане:\nПерейти в Потребление (1) → Начисление (2) Настроить интересующий период: В списке в колонке Деталь\xa0→ параметр Абон.плата по тарифному плану, в колонке Продукты\xa0→ Название тарифа.'],
 ['Сколько стоит тариф Премиум??',
  '## Другие Стоимостные Параметры\n\n| Смены тарифа первый раз в течение 30 дней                                                       | 0 ₽   |\n|-------------------------------------------------------------------------------------------------|-------|\n| Смены тарифа во 2-й и более раз в течение 30 дней                                               | 150 ₽ |\n| Первоначальный платеж при подключении в ССМ. Поступает на счет на оплату услуг связи, кроме 100 |       |\n| руб (плата за активацию)                                                                        |       |\n| 700 ₽                                                                                           |       |\n\n## Комментарий К Стоимости\n\nСтоимость услуг связи при нахождении в Крыму, ЛНР, ДНР, Луганской и Херсонской областях, а также в роуминге\xa0смотри в разделе **Путешествия**. Новым клиентам: **с абонементом** выгоднее (действует на все тарифы, кроме Премиум)! Выгодный Домашний интернет\xa0и ТВ: Смотрёшка,\xa0Wink\xa0\xa0, Мобильный домашний интернет Вызовы:\n устройства (автоинформатора). Порог отключения - 0 руб.\n\nИнтернет:\nИнтервал тарификации - 250 КБ, первая Интернет-сессия в месяце округляется до 1 МБ. Раздача Wi-Fi (тетеринг) доступна. Скорость интернета зависит от местонахождения клиента.\xa0**В домашнем регионе и по России** скорость мобильного интернета **не ограничена**.\xa0\n\nИсключение поездка в\xa0Норильск\xa0или Таймырский\xa0МР скорость до 512 Кб и в Чукотскую\xa0АО скорость до 128\xa0Кб\n\n## Общее Описание\n\nВключено в АП не требует подключения Связь при минусе (3 дня)\nМегаСемья\xa0(5 устройств макс.)'],
 ['Сколько стоит тариф Премиум??',
  '## Главная - Тарифы - Vip Vip\nПереход\n*789*6  Краткое описание Для всех устройств.\xa0Абон. плата:\xa0**1200** ₽/30 дн. Интернет:\xa050 Гб/30 дн. Минуты:\xa0**1700** мин./30 дн. на все мобильные и городские номера России.\xa0SMS:\xa0**300** шт./30 дн. на все номера России.\xa0\n\n## Стоимость Абонентская Плата\n\n| При переходе на тариф и со 2-го месяца после подключения   | 1200 ₽   |\n|------------------------------------------------------------|----------|\n| / 30 дн.                                                   |          |\n| Списания при подключении в 1-й месяц                       |          |\n| Разовая АП на 1й и на 16 день после подключения в салоне   | 600 ₽    |\n| Ежедневная АП после подключения у дилера/агента            | 40 ₽     |\n| / сут.                                                     |          |\n\n## Входит В Тариф\n\nИнтернет Интернет-трафик\n50 Гб/ 30 дн.\nДополнительный пакет МегаСемья (при добавлении первого участника)\n80 Гб\n\nЗвонки (минуты) На все номера России (МегаФон, другие мобильные операторы, городские)\n1700 мин./ 30 дн.\n\nSMS SMS на номера всех операторов России\n300 шт./ 30 дн.\n\nДополнительный пакет МегаСемья (при добавлении первого участника)\n80 Гб Доступны участникам, если подключены организатором: Безлимит на мессенджеры, соцсети, видео, музыку и кинотеатры\n0 ₽\nСвязь в минусе (сервисы не требуют подключения, при положительном балансе расходуют трафик) Такси и навигация в минусе: Uber, Ситимобил, Яндекс Go, Яндекс Карты, Google Карты, Apple Maps, 2ГИС и MAPS.ME\n3 сут.\n\nМессенджеры в минусе: WhatsApp,Telegram, Viber, IMO, Tam-Tam, Snapchat\n3 сут.\n\nБезлимит на банки: Модульбанк, Альфа-Банк, Локо-Банк, Промсвязьбанк, Сбер, Тинькофф, Совкомбанк, Бланк, Делобанк, Просто Банк, Точка, ВТБ, Райффайзенбанк\n3 сут.\n\nМинуты на МегаФон по России в минусе\n3 сут.\n\nПолезные сервисы (не требуют подключения)'],
 ['Сколько стоит тариф Премиум??',
  '## Остальные МегаСилы\n\nPRE-5G\nАвтоматические настройки управления трафиком, позволяющие перераспределять\xa0трафик.\nМегаСила: *105*1969# и для остальных тарифов: *105*1994# Для ТП Закачайся: *105*1980#\nУлучшает\xa0качество\xa0и стабильность\xa0 мобильного интернета.\xa0 НАЗВАНИЕ КРАТКОЕ ОПИСАНИЕ\n ПЕРЕХОД\n Подключение\xa0-\xa00 руб.\xa0**Абонентская плата:** Подключении как МегаСила - 0 руб., Подключена как доп. опция расширитель на\xa0 тарифах с выбором МегаСил - 99 руб., На остальных тарифах от 99 руб., подробнее в описании. Абонентская плата для линейки "Закачайся":\xa0Подключении Переносит остатки пакетов минут / SMS / *105*1920# и *105*1969# для линейки с 06.12.2022 интернета, включенные в тарифный план на Перенос остатков для Без переплат и МегаФон 3.0 следующий расчётный период. Для линеек Без Переплат и МегаФон 3.0.\n*105*1945#\nБонусные ГБ\n Только для\xa0клиентов\xa0текущей линейки МегаФон 3.0.\xa0При подключении выдается дополнительный объем\xa0интернета, который обновляется каждые 30 дней. Доступно на\xa0 смартфонах/телефонах/ планшетах, а также в модемах/ роутерах.\xa0Подключение:\xa00 ₽ \xa0Абон.плата:\xa00 ₽\n*105*1969#\nОбмен минут на ГБ\n МегаСила для тарифов линейки\xa0МегаФон\xa03.0. Подключение: 0 руб.\xa0На тарифах Минимум, Интернет: абон. плата - 69 ₽/30 дн.,\xa0курс обмена - 150. На тарифах\xa0МегаТариф, Максимум, VIP , Премиум\xa0-\xa00 ₽/30 дн.,\xa0курс обмена -\xa0300\nНАЗВАНИЕ\nКРАТКОЕ ОПИСАНИЕ Доступ к фильмам и сериалам в хорошем качестве (Full HD и 4K), просмотр на любых START для Без Переплат и МегаФон 3.0 устройствах, без рекламы.\xa0Подключение:\xa00 руб. Абон. плата: на тарифах линеек Без Переплат и МегаФон 3.0: Минимум,\xa0Интернет, Звонки, МегаТариф, Максимум, VIP,\xa0Премиум\xa0-\xa0**199 ₽/30 дн.**, при первом подключении\xa0START на номер -\xa0первые 30 дней бесплатно.\n\n## Дополнительные опции для линейки МегаФон'],
 ['Сколько стоит тариф Премиум??',
  '## Дополнительные опции для линейки МегаФон\n\nКРАТКОЕ ОПИСАНИЕ ПЕРЕХ\nОД\nНАЗВАНИЕ\nБезлимитные звонки на номера МегаФон России, не расходуют пакет минут, выданный в рамках тарифа.\xa0Подключение:\xa00 руб.\xa0Абон.плата: 169 ₽\n*105*19\n10#\nБезлимит на МегаФон для Без Переплат и МегаФон 3.0\nДоступно 300 SMS/мес. на все мобильные номера России для абонентов ТП линеек Без Переплат и МегаФон 3.0.\xa0Подключение: 0 руб.\xa0Абон.плата:\xa099 ₽\n*105*19\n04#\n+ 300 SMS/СМС для Без Переплат и МегаФон 3.0\nДоступно 300 минут на\xa0направление минут\xa0в рамках\xa0пакета трафика по тарифу. Только\xa0для линеек Без Переплат и МегаФон 3.0.\xa0Подключение: 0 руб.\xa0Абон.плата: 199 ₽\n*105*18\n99#\n+ 300 минут для Без Переплат и МегаФон 3.0\nДополнительно 3\xa0ГБ для\xa0ТП линеек Без Переплат и МегаФон 3.0.\xa0Подключение: 0 руб.\xa0Абон.плата:\xa099 ₽\n*105*19\n01#\n+ 3 ГБ\nДополнительно 5\xa0ГБ для\xa0линеек Без Переплат и МегаФон 3.0.\xa0Подключение\xa0-\xa00 руб.\xa0Абон.плата\xa0119 ₽\n*105*19\n02#\n+ 5 ГБ\nДополнительно 10\xa0ГБ для\xa0ТП линеек Без Переплат,\xa0МегаФон 3.0 и Персональный 3.0.\xa0Подключение:\xa00 руб.\xa0Абон.плата\xa0199 ₽\n*105*19\n21#\n+ 10 ГБ\nКРАТКОЕ ОПИСАНИЕ ПЕРЕХ\nОД\nНАЗВАНИЕ\nДополнительно 20\xa0ГБ для\xa0ТП линеек Без Переплат и МегаФон 3.0.\xa0Подключение\xa0-\xa00 руб.\xa0Абон.плата\xa0299 ₽\n*105*19\n33#\n+ 20 ГБ для Без переплат и МегаФон 3.0\n\n## Персональные МегаСилы\nПерсональные МегаСилы на выбор Непубличные, предлагаются по персональному предложению в МЛК. Доступны не всем клиентам Не подключаются автоматически при подключении или переходе Клиент может выбрать и подключить\xa0МегаСилы самостоятельно в ЛК/ МЛК На каждом тарифе доступен определенный лимит МегаСил ( доступность самого лимита определяется индивидуально) Подключение МегаСил сверх лимита -\xa0недоступно При исчерпании основного пакета\xa0подключенные МегаСилы продолжают работать\n\n\n## Премиум Пакеты на интернет, звонки, SMS'],
 ['Сколько стоит тариф Премиум??',
  '## Общее Описание\n\nВключено в АП не требует подключения Связь при минусе (3 дня)\nМегаСемья\xa0(5 устройств макс.)\n\nПолезные сервисы\nМинуты на все номера России\xa0\n\xa0Бесплатных устройств: 2\n\xa0VIP-статус "Золото"\nМинуты на МегаФон РФ\nSMS, Интернет\n+80ГБ для МегаСемьи\nТакси и навигация\nУчастникам доступны\nМессенджеры\nБезлимит на мессенджеры\nБанки\nБезлимит на музыку и кинотеатры Безлимит на соцсети Безлимит на видео\nДоп. опции требуют подключения\nМегаСилы требуют подключения\nПерсональные МегаСилы требуют подключения\nКлиент может выбрать:\nКлиент может выбрать:\nPre 5G\nStart\n6\n3\nБезлимит на музыку и кинотеатры\nБесплатные\nБезлимит на МегаФон\nБезлимит на соцсети\n Перенос остатков\n Pre 5G\nБезлимит на видео\nБонусные ГБ\nБезлимитный интернет Безлимитные SMS\nБезлимит на мессенджеры\nБезлимит на мессенджеры\nЯндекс Плюс МегаФон\nМинуты на все номера России +300минут\nБезлимит на соцсети\nБезлимит на МегаФон\nБезлимит на видео\nИнтернет +3ГБ,\xa0+5ГБ,\xa0+15ГБ\n SMS +60 SMS\nОбмен минут на ГБ\xa0\n Интернет +3ГБ,\xa0+5ГБ,\xa0+10ГБ,+20ГБ\nБезлимит на МегаФон\nМинуты +300минут\n SMS +300SMS\nБезлимит на музыку и кинотеатры\nПлатные\n Start\nУправление тарифом\n КЛИЕНТ:\n СОТРУДНИК:\n Подключение:\xa0\n Доступно в каналах: КЦ, ССМ, ФС, ПМ VIP\n Консультация - Выполняется Действие\xa0- ССМ-портал\n\nЛК/МЛК: USSD: *789*6# IVR:\xa0вызов на номер\xa005007897 SMS: любой текст на номер\xa005007897\nВходит в тариф (не требуют подключения):\n Пакеты интернет, минут, SMS, Связь в минусе, Включенные сервисы Основные услуги и сервисы. Не требуют подключения и дополнительной оплаты.\n\nПакеты'],
 ['Сколько стоит тариф Премиум??',
  'Абонентская плата списывается\xa0 на 10 минут раньше начала расчётного периода и фактического предоставления пакетов.\n\nобновлены только через 10 минут после списания. Пакеты предоставляются с точностью до минуты по времени подключения/перехода на ТП.\n\n## Дополнительная абонентская плата:\nПокупая SIM-карту у агента или мультибренда (Связной, МВидео и т.д), клиент соглашается с условиями возможного дополнительного списания АП при выезде за пределы домашнего региона в первые 60 дней с момента регистрации SIM-карты. Находясь вне домашнего региона более 30 дней начинает списываться АП\xa0(реализовано в виде опции **Дополнительная абонентская плата за несоблюдение условий Акции**)\n## Для новых клиентов (при новом подключении)\n АП за тариф списывается только при\xa0 LC-статусе «Активен». В день подключения:\xa0АП списывается **разово** за первые 15 дней.\xa0 Пакеты предоставляются в полном объеме на 30 дней. Через\xa0 15 суток:\xa0АП\xa0списывается единовременно за следующие 15 суток, в том числе в минус. Со второго месяца:\xa0АП\xa0списывается полностью,\xa0**в том числе в минус**.\xa0 Списание происходит раз в 30 суток с момента подключения на тариф. В случае неоплаты период по тарифу\xa0 не сдвигается.\n\n| Сразу за 15 дней   | Раз в 30 дней   | За следующие 15 дней   |\n|--------------------|-----------------|------------------------|\n| 1-й день           | 15-й день       | 30-й день              |\n\n## Для новых клиентов (при новом подключении через дилера/агента)\nАП за ТП списывается только при\xa0LC-статусе «Активен». В первый месяц\xa0АП списывается\xa0**посуточно.**\xa0 Пакеты предоставляются в полном объеме.\xa0 При недостаточном\xa0 балансе АП однократно\xa0 списывается в минус, пакеты минут и ПД не предоставляются, доступ в интернет блокируется. Со второго месяца:\xa0АП\xa0списывается полностью,\xa0**в том числе в минус**.\xa0Списание происходит раз в 30 суток с момента подключения на тариф. В случае неоплаты период по тарифу\xa0не сдвигается.\n\nПосуточное списание платы Раз в 30 дней\n\n30-Й День 1-Й День'],
 ['Сколько стоит тариф Премиум??',
  '## Отображение в ССМ-портале\n1.\xa0Пакет Автопродления отображается на\xa0**Дашборде.**\xa0Для\xa0проверки остатка интернета кликни на пакет:\xa0Дашборд\xa0→ Дополнительный пакет интернета.\n\n\xa0\xa0\n2. В новом окне появится информация 3. Для **проверки наличия пакета** (2) пройди по пути:\xa0Левое меню\xa0→\xa0Сводка\xa0→ Продукты → Пакеты (1) Алгоритм проверки потребления/остатков дополнительного пакета Пройди по пути:\xa0**Левое меню\xa0→ Потребление\xa0→ Скидки абонента**\xa0(1) \nПроверка корректности начислений\n\n1. Пройди по пути: **Левое меню\xa0→ Потребление → Вызовы** (1) 2. Проверь **дату совершения вызова** (2)\xa0и **дату учета на балансе** (3)\n3. Дата и время\xa0**совершения вызова** должны быть ранее даты и времени\xa0**обновления тарифа** (даты списания АП)\n\n## МегаСилы и Доп. опции (требуют подключения):\nКлиент может выбрать и подключить МегаСилы Основные МегаСилы на выбор МегаСилы/Расширители не подключаются автоматически при подключении или переходе, Клиент может выбрать и подключить любые МегаСилы/Расширители\xa0 самостоятельно в ЛК/ МЛК или через USSD, а также при обращении в КЦ. На каждом тарифе доступен определенный лимит МегаСил Подключение МегаСил сверх лимита - недоступно. Клиент может подключить аналог МегаСилы (расширитель) - платно. За исключением "Бонусных ГБ", "Переноса остатков" и "Обмена трафика" - эти МегаСилы недоступны платно. При исчерпание основного пакета\xa0 подключенные МегаСилы продолжают работать.\xa0\n\nТари ф\nМегаС илы\xa0\nPre 5G\nБонус ные ГБ\nОбмен минут на ГБ\nST AR T\nБезлими т на соц.сети\nБезлимит на музыку и кинотеатры\nБезлим ит на видео\nБезлимит на МегаФон\nПерено с остатко в\nБезлимит на мессендже ры\n3\nМин имум\n3\nИнте рнет\n4\nМега Тари ф\n5\nМакс имум VIP\n6 10\nПрем иум\n\n##  Безлимиты: зануляют трафик на сервисы'],
 ['Сколько стоит тариф Премиум??',
  'Минуты на МегаФон по России в минусе\n3 сут.\n\nПолезные сервисы (не требуют подключения)\n\n| VIP статус                                                            | 0 ₽   |\n|-----------------------------------------------------------------------|-------|\n| Безлимит на МегаФон по России (при исчерпании основного пакета минут) | 0 ₽   |\n| Полезные сервисы (требуют подключения)                                |       |\n| Виртуальный помощник Ева                                              | 0 ₽   |\n\nДоступные МегаСилы и Доп. опции на тарифе (требуют подключения, не расходуют интернет-трафик)\n\n| МегаСилы                | 6   |\n|-------------------------|-----|\n| Перенос остатков        | 0 ₽ |\n| / 30 дн.                |     |\n| Бонусные ГБ             | 0 ₽ |\n| / 30 дн.                |     |\n| Обмен 300 минут на 5 ГБ | 0 ₽ |\n| / 30 дн.                |     |\n\nPre-5G 0 ₽ / 30 дн.\n\n99 ₽ / 30 дн.\n\nМегаСила (на выбор в рамках лимита) (прочерк = недоступна)\nДоп.опции (и МегаСилы сверх лимита) (прочерк = недоступна)\nБезлимит на соц.сети: ВК, ОК, TikTok\n\n0 ₽ / 30 дн.\n69 ₽ / 30 дн.\n\nМегаСила (на выбор в рамках лимита) (прочерк = недоступна)\nДоп.опции (и МегаСилы сверх лимита) (прочерк = недоступна)\nБезлимит на музыку и кинотеатры: Я.Музыка, Apple Music, Звук, VK Музыка, ИВИ, OKKO, START, Кинопоиск, Амедиатека, PREMIER 0 ₽ / 30 дн.\n\n99 ₽ / 30 дн.\n\nМегаСила (на выбор в рамках лимита) (прочерк = недоступна)\nДоп.опции (и МегаСилы сверх лимита) (прочерк = недоступна)\nБезлимит на видео: YouTube, RuTube'],
 ['Сколько стоит тариф Премиум??',
  '##  Безлимиты: зануляют трафик на сервисы\n\nНазвание в ССМ- портале\nСервисы\nUSSD для управления опцией\n WhatsApp, Viber, Telegram, ТамТам, Snapchat, IMO\n*105*1969#\xa0\nБезлимит на мессенджеры Безлимит на соцсети\n ВКонтакте, Одноклассники, TikTok\n*105*1969# *105*2023#\nБезлимит на музыку\xa0и кинотеатры\n Яндекс Музыка, Apple Music, Звук, VK\xa0Музыка, ИВИ, OKKO, START, Кинопоиск, Амедиатека, PREMIER\nБезлимит на видео\nYouTube, RuTube\n*105*1969# *105*1969#\nБезлимит на МегаФон\nНеограниченные звонки на\xa0номера МегаФона России. Не\xa0расходуют пакет минут по\xa0тарифу.\n\nПодключаются только при положительном балансе. Не расходуют пакет интернет-трафика при использовании приложений и браузера. При использовании VPN безлимитные сервисы недоступны. Не работают при минусе. Не работают в роуминге. Работают во всех устройствах. Отключаются при переходе на другой тариф. МегаСила не\xa0совместима с аналогичной доп. опцией (расширителем)\xa0и наоборот. следующего периода.\n\n## Остальные МегаСилы']]

## Конвертация

In [6]:
ort_model = ORTModelForSequenceClassification.from_pretrained(ORIGINAL_MODEL_PATH, export=True)
tokenizer = AutoTokenizer.from_pretrained(ORIGINAL_MODEL_PATH)

Framework not specified. Using pt to export the model.
Using the export variant default. Available variants are:
    - default: The default ONNX variant.

***** Exporting submodel 1/1: XLMRobertaForSequenceClassification *****
Using framework PyTorch: 2.4.0+cu121
Overriding 1 configuration item(s)
	- use_cache -> False
Saving external data to one file...


In [7]:
optimizer = ORTOptimizer.from_pretrained(ort_model)

In [3]:
optimization_config = OptimizationConfig(
    disable_attention=False,
    disable_attention_fusion=False,
    disable_bias_gelu=False,
    disable_bias_gelu_fusion=False,
    disable_bias_skip_layer_norm=False,
    disable_bias_skip_layer_norm_fusion=False,
    disable_embed_layer_norm=False, 
    disable_embed_layer_norm_fusion=False,  
    disable_gelu=False,
    disable_gelu_fusion=False,
    disable_group_norm_fusion=False,  
    disable_layer_norm=False,
    disable_layer_norm_fusion=False,
    disable_packed_kv=False,
    disable_rotary_embeddings=False,
    disable_shape_inference=False,
    disable_skip_layer_norm=False,
    disable_skip_layer_norm_fusion=False,
    enable_gelu_approximation=True,
    enable_gemm_fast_gelu_fusion=False,  
    enable_transformers_specific_optimizations=True,
    fp16=True,
    no_attention_mask=False,
    optimization_level=99,
    optimize_for_gpu=True,
    optimize_with_onnxruntime_only=False,
    use_mask_index=False,
    use_multi_head_attention=True,
    use_raw_attention_mask=False  
)

In [9]:
optimizer.optimize(save_dir=ONNX_MODEL_PATH,
                   optimization_config=optimization_config,
                   use_external_data_format=True,
                   one_external_file=True)

The argument use_external_data_format in the ORTOptimizer.optimize() method is deprecated and will be removed in optimum 2.0.
Optimizing model...
Failed to run symbolic shape inference. Please file an issue in https://github.com/microsoft/onnxruntime.
Configuration saved in C:\RAG\bge-reranker-v2-m3\onnx\bge-reranker-v2-m3-onnx-gpu\ort_config.json
Optimized model saved at: C:\RAG\bge-reranker-v2-m3\onnx\bge-reranker-v2-m3-onnx-gpu (external data format: True; saved all tensor to one file: True)


WindowsPath('C:/RAG/bge-reranker-v2-m3/onnx/bge-reranker-v2-m3-onnx-gpu')

## Сравнение с оригинальной моделью

In [7]:
from FlagEmbedding import FlagReranker

In [8]:
flag_model_bge_reranker = FlagReranker(ORIGINAL_MODEL_PATH, use_fp16=True)

In [9]:
sorted(flag_model_bge_reranker.compute_score(test_queries, normalize=True))

[0.0025409926265789833,
 0.002704429369409325,
 0.010860331013277533,
 0.014728613849722676,
 0.03143985047843082,
 0.06430005451227754,
 0.07821887082692507,
 0.14584377135587895,
 0.20914644271566601,
 0.3622920679980696]

## Интерфейс для загрузки и вызова конвертированной модели

In [19]:
# import numpy as np
# import torch
# from transformers import AutoTokenizer
# import onnxruntime as ort
# from typing import List, Optional
# import timeit

# class ORTFEDenseModelInterface:
#     def __init__(
#         self,
#         session: ort.InferenceSession,
#         tokenizer: AutoTokenizer,
#         use_fp16=True,
#         normalize=True,
#         device="cuda",
#     ):
#         self.session = session
#         self.tokenizer = tokenizer
#         self.use_fp16 = use_fp16
#         self.normalize = normalize
#         self.device = device

#         # Получите правильное имя выходного тензора
#         self.output_name = self.session.get_outputs()[0].name

#     @classmethod
#     def from_pretrained(
#         cls,
#         model_path: str,
#         tokenizer_path: str,
#         use_fp16: bool = True,
#         normalize: bool = True,
#         device: str = "cuda",
#     ):
#         # Настройка провайдера CUDA
#         providers = [('CUDAExecutionProvider', {
#             'device_id': 0,
#             'arena_extend_strategy': 'kSameAsRequested',
#             'gpu_mem_limit': 5 * 1024 * 1024 * 1024,  # 5 GB
#             'cudnn_conv_algo_search': 'EXHAUSTIVE',
#             'do_copy_in_default_stream': True,
#         })]
        
#         # Загрузить сессию ONNX
#         so = ort.SessionOptions()
#         so.enable_mem_pattern = False
#         so.enable_mem_reuse = False
        
#         # Включить уменьшение размера арены памяти
#         so.add_session_config_entry("memory.enable_memory_arena_shrinkage", "cpu:0; gpu:0")
#         so.add_session_config_entry('session.use_device_allocator_for_initializers', "1")
#         so.execution_mode = ort.ExecutionMode.ORT_SEQUENTIAL
#         so.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL

#         session = ort.InferenceSession(model_path, providers=providers, sess_options=so)

#         return cls(
#             session=session,
#             tokenizer=AutoTokenizer.from_pretrained(tokenizer_path, use_fast=True),
#             use_fp16=use_fp16,
#             normalize=normalize,
#             device=device,
#         )

#     def _sigmoid(self, x: np.ndarray) -> np.ndarray:
#         return 1 / (1 + np.exp(-x))
        
#     @torch.no_grad()
#     def compute_score(
#         self,
#         sentence_pairs: List[List[str]],
#         tokenizer_padding: bool = True,
#         tokenizer_truncation: bool = True,
#         max_length: int = 8192,
#         batch_size: int = 12,
#         normalize: Optional[bool] = None,
#     ):
#         all_logits = []
#         normalize = self.normalize if normalize is None else normalize

#         for idx in range(0, len(sentence_pairs), batch_size):
#             batch = sentence_pairs[idx:idx+batch_size]
#             inputs = self.tokenizer(
#                 batch,
#                 padding=tokenizer_padding,
#                 truncation=tokenizer_truncation,
#                 return_tensors="pt",
#                 max_length=max_length,
#             )

#             # Переместить входные данные на device
#             inputs = {k: v.to(self.device) for k, v in inputs.items()}
            
#             # Преобразовать входные данные в тензоры PyTorch на GPU
#             ort_inputs = {}
#             for name, value in inputs.items():
#                 torch_tensor = value.to(dtype=torch.int64, device=self.device).contiguous()
#                 ort_inputs[name] = torch_tensor

#             # Создать IOBinding для батча
#             io_binding = self.session.io_binding()

#             # Привязать входные данные
#             for name, torch_tensor in ort_inputs.items():
#                 io_binding.bind_input(
#                     name=name,
#                     device_type=self.device,
#                     device_id=0,
#                     element_type=np.int64,
#                     shape=tuple(torch_tensor.shape),
#                     buffer_ptr=torch_tensor.data_ptr(),
#                 )

#             # Привязать выходные данные к устройству, пусть ORT выделит тензор вывода
#             io_binding.bind_output(
#                 name=self.output_name,
#                 device_type=self.device,
#                 device_id=0
#             )

#             # Выполнить инференс
#             self.session.run_with_iobinding(io_binding)

#             # Получить выходное значение OrtValue
#             ortvalue = io_binding.get_outputs()[0]

#             # Преобразовать OrtValue в массив NumPy
#             output_numpy = ortvalue.numpy()

#             # Обработать выходной тензор
#             logits = output_numpy[:, 0]

#             if normalize:
#                 logits = self._sigmoid(logits)

#             all_logits.append(logits)
            
#              # Очистка данных
#             del inputs, ort_inputs, ortvalue, output_numpy, logits
#             io_binding.clear_binding_inputs()
#             io_binding.clear_binding_outputs()
#             torch.cuda.empty_cache()

#         # Объединить результаты
#         all_logits = np.concatenate(all_logits, axis=0)

#         # Вернуть результат в виде списка
#         return all_logits.tolist()

In [20]:
# # Использование
# tokenizer_path = r"C:\RAG\bge-reranker-v2-m3\onnx\bge-reranker-v2-m3-onnx-gpu"
# model_path = f"{tokenizer_path}\model_optimized.onnx"

In [21]:
# ort_model_interface = ORTFEDenseModelInterface.from_pretrained(
#     model_path=model_path,
#     tokenizer_path=tokenizer_path,
#     use_fp16=False,
#     normalize=True,
#     device="cuda"
# )

In [10]:
from typing import Any, List, Optional
import timeit
import numpy as np
import torch
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForSequenceClassification

import os
os.environ['CUDA_VISIBLE_DEVICES']="0"


class ORTFEDenseModelInterface:
    def __init__(
        self,
        model: ORTModelForSequenceClassification = None,
        tokenizer: AutoTokenizer = None,
        use_fp16=True,
        normalize=True,
        device="cuda",
    ):
        self.model = model
        self.tokenizer = tokenizer
        self.use_fp16 = use_fp16
        self.normalize = normalize
        self.device = device

    def load(
        self,
        path: str,
    ):
        self.tokenizer = AutoTokenizer.from_pretrained(path, use_fast=True)
        self.model = ORTModelForSequenceClassification.from_pretrained(path)
        self.model.to(self.device)

    @classmethod
    def from_pretrained(
        cls,
        path: str,
        normalize: bool = True,
        device: str = "cuda",
    ):
        model = ORTModelForSequenceClassification.from_pretrained(path)
        model.to(device)
        return cls(
            model=model,
            tokenizer=AutoTokenizer.from_pretrained(path, use_fast=True),
            normalize=normalize,
            device=device,
        )

    def _sigmoid(self, x: torch.Tensor) -> torch.Tensor:
        return 1 / (1 + torch.exp(-x))

    @torch.no_grad()
    def compute_score(
        self,
        sentence_pairs: List[List[str]],
        tokenizer_padding: bool = True,
        tokenizer_truncation: bool = True,
        max_length: int = 8192,
        batch_size: int = 12,
        normalize: Optional[bool] = None,
    ):
        all_logits = []
        normalize = self.normalize if normalize is None else normalize
        
        for i in range(0, len(sentence_pairs), batch_size):
            batch = sentence_pairs[i:i+batch_size]
            inputs = self.tokenizer(
                batch,
                padding=tokenizer_padding,
                truncation=tokenizer_truncation,
                return_tensors="pt",
                max_length=max_length,
            )

            # Перемещаем входные данные на GPU
            inputs = {k: v.to(self.device) for k, v in inputs.items()}

            # Выполняем инференс
            outputs = self.model(**inputs)
            logits = outputs.logits[:, 0]

            if self.normalize:
                logits = self._sigmoid(logits)

            # Переносим результаты на CPU и конвертируем в numpy
            logits = logits.cpu().numpy()

            all_logits.append(logits)

            # Очистка памяти GPU
            del inputs, outputs
            torch.cuda.empty_cache()

        # Объединяем результаты
        result = np.concatenate(all_logits, axis=0)

        return result.tolist()

In [11]:
# Использование
model_path_onnx = r"C:\RAG\bge-reranker-v2-m3\onnx\bge-reranker-v2-m3-onnx-gpu"
ort_model_interface = ORTFEDenseModelInterface(normalize=True, device="cuda")
# Загружаем модель и токенизатор
ort_model_interface.load(path=model_path_onnx)

use_io_binding was set to False, setting it to True because it can provide a huge speedup on GPUs. It is possible to disable this feature manually by setting the use_io_binding attribute back to False.


In [ ]:
# убедится что включился use_io_binding выше

In [12]:
results = ort_model_interface.compute_score(test_queries)
sorted(results)

[0.001573112211190164,
 0.002736221067607403,
 0.01358439214527607,
 0.03173859417438507,
 0.06441766768693924,
 0.0755847916007042,
 0.13939638435840607,
 0.14475227892398834,
 0.2230384200811386,
 0.3413628935813904]

In [13]:
# тестирование
import random
import timeit

# Функция для первого варианта
def variant1():
    results = ort_model_interface.compute_score(test_queries, batch_size=32)
    return results

# Функция для второго варианта
def variant2():
    results = flag_model_bge_reranker.compute_score(test_queries, batch_size=32, max_length=1024,  normalize=True)
    return results

# Количество повторений для более точного измерения
number = 20

# Измерение времени выполнения первого варианта
time1 = timeit.timeit(variant1, number=number)

# Измерение времени выполнения второго варианта
time2 = timeit.timeit(variant2, number=number)

print(f"Время выполнения первого варианта: {time1:.6f} секунд")
print(f"Время выполнения второго варианта: {time2:.6f} секунд")

# Сравнение производительности
if time1 < time2:
    print(f"Первый вариант быстрее на {(time2 - time1) / time2 * 100:.2f}%")
else:
    print(f"Второй вариант быстрее на {(time1 - time2) / time1 * 100:.2f}%")

# Проверка корректности результатов и расчет разницы
results1 = np.array(variant1())
results2 = np.array(variant2())

if np.allclose(results1, results2, rtol=1e-5, atol=1e-8):
    print("Результаты обоих вариантов практически идентичны")
else:
    print("Внимание: результаты вариантов различаются")
    
    # Расчет абсолютной разницы
    abs_diff = np.abs(results1 - results2)
    
    print(f"Максимальная абсолютная разница: {np.max(abs_diff):.6e}")
    print(f"Средняя абсолютная разница: {np.mean(abs_diff):.6e}")
    print(f"Медианная абсолютная разница: {np.median(abs_diff):.6e}")
    
    # Расчет относительной разницы
    rel_diff = np.abs((results1 - results2) / np.maximum(np.abs(results1), np.abs(results2)))
    
    print(f"Максимальная относительная разница: {np.max(rel_diff):.6%}")
    print(f"Средняя относительная разница: {np.mean(rel_diff):.6%}")
    print(f"Медианная относительная разница: {np.median(rel_diff):.6%}")

Время выполнения первого варианта: 3.077479 секунд
Время выполнения второго варианта: 5.040924 секунд
Первый вариант быстрее на 38.95%
Внимание: результаты вариантов различаются
Максимальная абсолютная разница: 3.080546e-03
Средняя абсолютная разница: 8.179604e-04
Медианная абсолютная разница: 6.161128e-04
Максимальная относительная разница: 1.547958%
Средняя относительная разница: 0.944112%
Медианная относительная разница: 0.906940%


## Сжатие в архив

In [45]:
!tar -czvf ../bge-reranker-v2-m3-onnx-gpu.tar.gz "C:\RAG\bge-reranker-v2-m3\onnx\bge-reranker-v2-m3-onnx-gpu"

tar: Removing leading drive letter from member names
a RAG/bge-reranker-v2-m3/onnx/bge-reranker-v2-m3-onnx-gpu
a RAG/bge-reranker-v2-m3/onnx/bge-reranker-v2-m3-onnx-gpu/config.json
a RAG/bge-reranker-v2-m3/onnx/bge-reranker-v2-m3-onnx-gpu/model_optimized.onnx
a RAG/bge-reranker-v2-m3/onnx/bge-reranker-v2-m3-onnx-gpu/model_optimized.onnx.data
a RAG/bge-reranker-v2-m3/onnx/bge-reranker-v2-m3-onnx-gpu/ort_config.json
a RAG/bge-reranker-v2-m3/onnx/bge-reranker-v2-m3-onnx-gpu/sentencepiece.bpe.model
a RAG/bge-reranker-v2-m3/onnx/bge-reranker-v2-m3-onnx-gpu/special_tokens_map.json
a RAG/bge-reranker-v2-m3/onnx/bge-reranker-v2-m3-onnx-gpu/tokenizer.json
a RAG/bge-reranker-v2-m3/onnx/bge-reranker-v2-m3-onnx-gpu/tokenizer_config.json


In [13]:
torch.cuda.empty_cache()

In [6]:
ort_model_interface.compute_score(test_queries, batch_size=32)

TypeError: 'NoneType' object is not callable